In [3]:
import pandas as pd
import numpy as np
import wfdb
import ast
from pathlib import Path

In [4]:
def load_raw_data(df, sampling_rate, path):
    if sampling_rate == 100:
        data = [wfdb.rdsamp(path+f) for f in df.filename_lr]
    else:
        data = [wfdb.rdsamp(path+f) for f in df.filename_hr]
    data = np.array([signal for signal, meta in data])
    return data

In [5]:
path = str(Path("C:/Users/DARYA/Desktop/Study/yp20/dataset/").resolve()) + r'/'

sampling_rate=100

# load and convert annotation data
Y = pd.read_csv(path+'ptbxl_database.csv', index_col='ecg_id')
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

# Load raw signal data
X = load_raw_data(Y, sampling_rate, path)

# Load scp_statements.csv for diagnostic aggregation
agg_df = pd.read_csv(path+'scp_statements.csv', index_col=0)
agg_df = agg_df[agg_df.diagnostic == 1]


In [6]:
def aggregate_diagnostic(y_dic):
    tmp = []
    for key in y_dic.keys():
        if key in agg_df.index:
            tmp.append(agg_df.loc[key].diagnostic_class)
    return list(set(tmp))

# Apply diagnostic superclass
Y['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_diagnostic)


In [ ]:
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    print(Y.describe(include='all'))
    print(Y.head())



          patient_id           age           sex       height       weight  \
count   21799.000000  21799.000000  21799.000000  6974.000000  9421.000000   
unique           NaN           NaN           NaN          NaN          NaN   
top              NaN           NaN           NaN          NaN          NaN   
freq             NaN           NaN           NaN          NaN          NaN   
mean    11250.156521     62.769301      0.479150   166.702323    70.995223   
std      6235.026404     32.308813      0.499577    10.867321    15.878803   
min       302.000000      2.000000      0.000000     6.000000     5.000000   
25%      5974.500000     50.000000      0.000000   160.000000    60.000000   
50%     11419.000000     62.000000      0.000000   166.000000    70.000000   
75%     16607.500000     72.000000      1.000000   174.000000    80.000000   
max     21797.000000    300.000000      1.000000   209.000000   250.000000   

               nurse          site      device       recording_

In [12]:
Y.dtypes

patient_id                      float64
age                             float64
sex                               int64
height                          float64
weight                          float64
nurse                           float64
site                            float64
device                           object
recording_date                   object
report                           object
scp_codes                        object
heart_axis                       object
infarction_stadium1              object
infarction_stadium2              object
validated_by                    float64
second_opinion                     bool
initial_autogenerated_report       bool
validated_by_human                 bool
baseline_drift                   object
static_noise                     object
burst_noise                      object
electrodes_problems              object
extra_beats                      object
pacemaker                        object
strat_fold                        int64


patient_id - id пациента
age - возраст
sex - пол
height - рост
weight - вес
nurse - номер медсестры
site - место записи
device - устройство ЭКГ
recording_date - Дата и время записи ЭКГ
report - отчет кардиолога по ЭКГ
scp_codes - SCP коды
heart_axis - электрическая ось сердца
infarction_stadium1 - стадия инфаркта
infarction_stadium2 - вторая стадия инфаркта
validated_by - проверено кардиологом
second_opinion - есть ли второе другое мнение
initial_autogenerated_report - автоматически сгенерированный отчет с помощью ус-ва ЭКГ
validated_by_human - проверено человеком
baseline_drift - присутствует дрейф или скачок базовой линии
static_noise - электрический/статический шум
burst_noise - взрывной шум
electrodes_problems - проблемы с электродами
extra_beats - дополнительные удары
pacemaker - кардиостимулятор
strat_fold - предложенные стратифицированные складки
filename_lr - название файла с данными на частоте 100 ГЦ
filename_hr - название файла с данными на частоте 500 ГЦ
diagnostic_superclass - аббревиатуры диагнозов

In [71]:
Y['diagnostic_superclass'][2]
a = []
b = 0
d = {'H':0, 'C':0,'M':0,'N':0,'S':0,}
for i in Y.index:
    a += Y['diagnostic_superclass'][i]
    if 'HYP' in Y['diagnostic_superclass'][i]:
        d['H'] = d['H'] + 1
    if 'CD' in Y['diagnostic_superclass'][i]:
        d['C'] += 1
    if 'MI' in Y['diagnostic_superclass'][i]:
        d['M'] += 1
    if 'NORM' in Y['diagnostic_superclass'][i]:
        d['N'] += 1
    if 'STTC' in Y['diagnostic_superclass'][i]:
        d['S'] += 1

b

0

In [88]:
Y['scp_codes'][i].keys()

s = []
for i in Y.index:
    s += Y['scp_codes'][i].keys()

list(set(s))

['LVH',
 'NT_',
 'PSVT',
 'BIGU',
 'AMI',
 'ISC_',
 'TRIGU',
 'ILBBB',
 'INJIN',
 'EL',
 'ISCAS',
 'ISCAL',
 'SBRAD',
 'PAC',
 'ISCAN',
 'INJAS',
 'VCLVH',
 '3AVB',
 'PVC',
 'INVT',
 'QWAVE',
 'SVTAC',
 'AFIB',
 'RVH',
 'TAB_',
 'SEHYP',
 'LAO/LAE',
 'PMI',
 'WPW',
 'LVOLT',
 '1AVB',
 'LPFB',
 'ASMI',
 'ISCIL',
 'SARRH',
 'ISCLA',
 'ISCIN',
 'NDT',
 'LNGQT',
 'IMI',
 'PRC(S)',
 'RAO/RAE',
 'STE_',
 'SR',
 'STACH',
 'CRBBB',
 'ALMI',
 'HVOLT',
 'CLBBB',
 'NORM',
 'PACE',
 'LOWT',
 'INJLA',
 'ABQRS',
 'STD_',
 'LPR',
 'INJAL',
 'IPLMI',
 'ANEUR',
 'NST_',
 '2AVB',
 'SVARR',
 'ILMI',
 'INJIL',
 'IVCD',
 'AFLT',
 'IRBBB',
 'IPMI',
 'LAFB',
 'LMI',
 'DIG']

In [ ]:
d

{'H': 2649, 'C': 4898, 'M': 5469, 'N': 9514, 'S': 5235}

In [89]:
Y['heart_axis'].unique()

array([nan, 'LAD', 'ALAD', 'RAD', 'AXR', 'MID', 'ARAD', 'AXL', 'SAG'],
      dtype=object)

Ось ЭКГ - главное направление общей электрической активности сердца:
'LAD' - левое (левая ось)
'ALAD' - Abnormal LAD, extreme left axis deviation
'RAD' - правое (правая ось)
'AXR' - Borderline right-axis deviation 	Vertical axis
'MID' - Normal axis
'ARAD' - Abnormal RAD, extreme right axis deviation
'AXL' - Borderline left-axis deviation
'SAG' - Saggital type (S1-S2-S3 Pattern)